# General

Specify file name and sampling period

In [9]:
filename_prefix = './data/arin/Omero19979_'
sampling_period = 5

%matplotlib

import importlib
importlib.reload(pipeline.vis)

Using matplotlib backend: Qt5Agg


<module 'pipeline.vis' from '/home/arin/phd/phd-time-series-pipeline/pipeline/vis.py'>

Main shebang (this probably isn't the purpose of a jupyter notebook)

In [10]:
#!/usr/bin/env python3
import os

import numpy as np
import scipy as sp
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import sklearn.metrics
import igraph as ig

import pipeline.dataexport
import pipeline.dataimport
import pipeline.periodogram
import pipeline.score
import pipeline.tsman
import pipeline.vis

import featext.tsman
import featext.graph
import featext.vis

import catch22
import leidenalg

def add_classicalAttr(cell, oversampling_factor = 1):
    """Computes classical periodogram and adds PdgramAttr attributes"""
    cell.flavin.classical.freqs, cell.flavin.classical.power = \
            pipeline.periodogram.classical(cell.time, cell.flavin.reading_processed,
                                oversampling_factor = oversampling_factor)

def add_bglsAttr(cell):
    """Computes BGLS and adds PdgramAttr attributes"""
    cell.flavin.bgls = pipeline.PdgramAttr()
    cell.flavin.bgls.label = 'Bayesian General Lomb-Scargle Periodogram'
    cell.flavin.bgls.power_label = 'Probability'
    err = np.ones(len(cell.flavin.reading_processed))*\
            np.sqrt(np.max(cell.flavin.reading_processed))
    cell.flavin.bgls.freqs, cell.flavin.bgls.power = \
            pipeline.periodogram.bgls(cell.time, cell.flavin.reading_processed, err,
                    plow = 30.0, phigh = 360.0, ofac = 5)

def add_autoregAttr(cell):
    """
    Computes autoregressive model-based periodogram and adds PdgramAttr
    attributes
    """
    cell.flavin.autoreg = pipeline.PdgramAttr()
    cell.flavin.autoreg.label = \
            'Autogressive Model-Based Periodogram (Jia & Grima, 2020)'
    cell.flavin.autoreg.power_label = 'Power'
    freq_npoints = 1000
    cell.flavin.autoreg.freqs, cell.flavin.autoreg.power = \
            pipeline.periodogram.autoreg(cell.time,
                                         cell.flavin.reading_processed,
                                         freq_npoints)

# FLAVIN: import data and process objects

# Import fluorescence info from CSVs
Dset_flavin = pipeline.dataimport.import_timeseries(
    filename_prefix+'flavin.csv', remain = 0.8)
# dummy so I get code to not complain; will be re-factored later
Dset_dcategory = [3] * len(Dset_flavin)
Dset_births = pipeline.dataimport.import_births(
    filename_prefix+'births.csv')

# Arranges information into DatasetAttr objects
Dset_data = pipeline.dataimport.CellAttr_from_datasets( \
        timeseries_df = Dset_flavin,
        categories_array = Dset_dcategory,
        births_df = Dset_births,
        sampling_pd = sampling_period)
Dset = pipeline.DatasetAttr(Dset_data)

# Add labels
strainlookup = pd.read_csv(filename_prefix+'strains.csv', \
                          index_col = 'position')
for ii, cell in enumerate(Dset.cells):
    cell.source = filename_prefix
    cell.medium.base = 'Delft'
    cell.medium.nutrients = {'glucose': 10}

    cell.strain = strainlookup.loc[cell.position].strain

    cell.flavin = pipeline.Fluo('flavin')
    cell.flavin.exposure = 60
    cell.flavin.reading = cell.y
    cell.flavin.category = Dset_dcategory[ii]


# mCherry: import data and process objects
try:
    Dset_mCherry_unsliced = pipeline.dataimport.import_timeseries(
        filename_prefix+'mCherry.csv', remain = 0.8)
    # restrict to cells with flavin readings
    idx_both = list(set(Dset_flavin.cellID) & set(Dset_mCherry_unsliced.cellID))
    Dset_mCherry = \
            Dset_mCherry_unsliced.loc[Dset_mCherry_unsliced.cellID.isin(idx_both)]

    # Arranges information into DatasetAttr objects
    # dummy -- will be better when I re-structure things... am just re-using a 
    # function for quick-and-dirty purposes, and it's obviously redundant
    mCherry_data = pipeline.dataimport.CellAttr_from_datasets( \
            timeseries_df = Dset_mCherry,
            categories_array = Dset_dcategory,
            births_df = Dset_births,
            sampling_pd = sampling_period)
    mCherry = pipeline.DatasetAttr(mCherry_data)
    mCherry_MATLABids = [cell.MATLABid for cell in mCherry.cells]

    # Add labels
    for ii, cell in enumerate(Dset.cells):
        cell.mCherry = pipeline.Fluo('mCherry')
        if cell.strain == 'htb2_mCherry_CRISPR':
            cell.mCherry.exposure = 100
        else:
            cell.mCherry.exposure = 0

        # loads in reading, cross-referencing by MATLABid.  This is awful, I know.
        if cell.MATLABid in mCherry_MATLABids:
            cell.mCherry.reading = \
                mCherry.cells[mCherry_MATLABids.index(cell.MATLABid)].y
except FileNotFoundError as error:
    print(error)
    print(f'No mCherry time series associated with this experiment: {filename_prefix}')

[Errno 2] No such file or directory: './data/arin/Omero19979_mCherry.csv'
No mCherry time series associated with this experiment: ./data/arin/Omero19979_


Define working dataset (list of cells)

In [11]:
Wlist = Dset.cells
#Wlist = [cell for cell in Dset.cells if cell.strain == 'FY4']
len(Wlist)

589

Optional: filters

In [ ]:
for cell in itertools.chain(Wlist):
    cell.flavin.reading_processed = \
            pipeline.tsman.stdfilter(cell.flavin.reading, Fs = 1/sampling_period)
pipeline.tsman.population_detrend(Wlist, 'flavin.reading_processed')

Optional: chop up time series

In [127]:
interval_start = 0
interval_end = 168

for cell in Wlist:
    cell.time = cell.time[interval_start:interval_end]
    cell.flavin.reading = cell.flavin.reading[interval_start:interval_end]

Optional: normalise for heatmap

In [40]:
for cell in Wlist:
    ts_mean = np.nanmean(cell.flavin.reading)
    ts_range = np.nanmax(cell.flavin.reading) - np.nanmin(cell.flavin.reading)
    cell.flavin.reading_processed = (cell.flavin.reading - ts_mean)/ts_range

Optional: add spectra

In [ ]:
for cell in Wlist:
    cell.flavin.reading_processed = cell.flavin.reading
    add_classicalAttr(cell, oversampling_factor = 1)
    #add_bglsAttr(cell)
    add_autoregAttr(cell)

General plotting

In [12]:
cell_index = 404
y_attr = 'flavin.reading'

Wlist[cell_index].plot_ts(y_attr=y_attr)
print(Wlist[cell_index].strain)
print(Wlist[cell_index].MATLABid)
print(Wlist[cell_index].position)

swe1_Del
405
20


In [ ]:
# With CSHL2021/BYG2021 colour palette
# (insert code here)

Kymograph

In [51]:
cell_attr = 'flavin.reading_processed'

pipeline.vis.kymograph(Wlist, cell_attr=cell_attr,
                      order_by='distfromcentre')

# Plots for presentation

Synchrony of YMCs

In [48]:
filename_prefix = './data/arin/Omero19972_'
# THEN RUN MAIN SHEBANG

In [50]:
Wlist = [cell for cell in Dset.cells if cell.strain == 'FY4']
len(Wlist[0].time)

264

In [7]:
# Glucose: 0 - 83
# Starvation: 84 - 180
# Recovery: 181 - 264

def kymograph_chopped(Wlist, interval_start, interval_end):
    # Chops time series
    for cell in Wlist:
        cell.time = cell.time[interval_start:interval_end]
        cell.flavin.reading = cell.flavin.reading[interval_start:interval_end]
    # Normalise for kymograph
    for cell in Wlist:
        ts_mean = np.nanmean(cell.flavin.reading)
        ts_range = np.nanmax(cell.flavin.reading) - np.nanmin(cell.flavin.reading)
        cell.flavin.reading_processed = (cell.flavin.reading - ts_mean)/ts_range
    # Draws kymograph
    pipeline.vis.kymograph(Wlist, cell_attr='flavin.reading_processed',
                          order_by='distfromcentre')

kymograph_chopped(Wlist, 181, 264)

In [51]:
# Spellman et al (1998)

def kymograph_chopped(Wlist, interval_start, interval_end):
    # Chops time series
    for cell in Wlist:
        cell.time = cell.time[interval_start:interval_end]
        cell.flavin.reading = cell.flavin.reading[interval_start:interval_end]
    # Normalise for kymograph
    for cell in Wlist:
        ts_mean = np.nanmean(cell.flavin.reading)
        cell.flavin.reading_processed = np.log2(cell.flavin.reading / ts_mean)
    # Draws kymograph
    pipeline.vis.kymograph(Wlist, cell_attr='flavin.reading_processed',
                          order_by='distfromcentre')

kymograph_chopped(Wlist, 84, 180)

<ipython-input-51-5c1c5d7812ad>:11: RuntimeWarning: invalid value encountered in log2
  cell.flavin.reading_processed = np.log2(cell.flavin.reading / ts_mean)


In [54]:
cell.flavin.reading
ts_mean = np.nanmean(cell.flavin.reading)
cell.flavin.reading_processed = np.log2(cell.flavin.reading / ts_mean)

In [56]:
fig, ax = plt.subplots()
ax.plot(cell.flavin.reading)
plt.show()

In [57]:
fig, ax = plt.subplots()
ax.plot(cell.flavin.reading_processed)
plt.show()

Causton strains

In [124]:
filename_prefix = './data/arin/Omero19979_'
# THEN RUN MAIN SHEBANG

In [120]:
# Chops time series - i.e. before glucose limitation
# WARNING: DO THIS ONCE
interval_start = 0
interval_end = 168
for cell in Dset.cells:
    cell.time = cell.time[interval_start:interval_end]
    cell.flavin.reading = cell.flavin.reading[interval_start:interval_end]
strain_list = ['swe1_Del', 'tsa1_Del_tsa2_Del', 'rim11_Del']
for strain in strain_list:
    # Choose cells corresponding to strain
    Wlist = [cell for cell in Dset.cells if cell.strain == strain]
    # Normalise for kymograph
    for cell in Wlist:
        ts_mean = np.nanmean(cell.flavin.reading)
        ts_range = np.nanmax(cell.flavin.reading) - np.nanmin(cell.flavin.reading)
        cell.flavin.reading_processed = (cell.flavin.reading - ts_mean)/ts_range
    # Draws kymograph
    pipeline.vis.kymograph(Wlist, cell_attr='flavin.reading_processed',
                          order_by='distfromcentre')

ZWF1

In [134]:
filename_prefix = './data/arin/Omero20016_'
# THEN RUN MAIN SHEBANG

In [123]:
# Chops time series - i.e. before glucose limitation
# WARNING: DO THIS ONCE
interval_start = 0
interval_end = 168
for cell in Dset.cells:
    cell.time = cell.time[interval_start:interval_end]
    cell.flavin.reading = cell.flavin.reading[interval_start:interval_end]
strain_list = ['zwf1_Del', 'by4741']
for strain in strain_list:
    # Choose cells corresponding to strain
    Wlist = [cell for cell in Dset.cells if cell.strain == strain]
    # Normalise for kymograph
    for cell in Wlist:
        ts_mean = np.nanmean(cell.flavin.reading)
        ts_range = np.nanmax(cell.flavin.reading) - np.nanmin(cell.flavin.reading)
        cell.flavin.reading_processed = (cell.flavin.reading - ts_mean)/ts_range
    # Draws kymograph
    pipeline.vis.kymograph(Wlist, cell_attr='flavin.reading_processed',
                          order_by='distfromcentre')